In [83]:
import sys
sys.path.append('..')
import numpy as np
import pandas as pd
import preprocessing.preprocessor
import importlib
importlib.reload(preprocessing.preprocessor)
from preprocessing.preprocessor import Preprocessor

In [119]:
series_train_set_ocr = pd.read_json('../../data/hackathon/train_set_ocr.json', typ='series')

In [120]:
df_train = series_train_set_ocr.reset_index()
df_train.rename(columns={'index': 'file', 0: 'text'}, inplace=True)

In [86]:
preprocessor = Preprocessor(df_train)

In [87]:
preprocessor.preprocess_text('text')

In [88]:
preprocessor.df

,file,text
0,test_hashed\pit37_v1\9d6f61c6-52ad-4546-85b7-b...,"pola lalwe poda podatnik pola , ciemne _ urzad..."
1,test_hashed\pit37_v1\f14ea197-70bd-45ae-9a22-d...,pltaxpl poua 'wypelnia podatnik pola ciemne wy...
2,test_hashed\pit37_v1\070a521a-7038-4551-bc6a-b...,mypfenl wypelnic dulymi daukowanluiteram clrny...
3,test_hashed\pit37_v1\f4d15b30-6e96-4452-b614-a...,pltaxpl pola jasne wypełnia podatnik _pola cie...
4,test_hashed\pit37_v1\64d085a8-39a0-4887-859e-8...,pitaxpl la jash€ wypelnia poditńik pcu ciemne ...
...,...,...
10879,./test_hashed/specification/8a1bb716-156c-4eca...,"msds 002z4 ello ~arte-vewaa @were hkc hm2 ""7c0..."
10880,./test_hashed/specification/e42169d6-f1c4-416a...,"june 1992 40 03-540 appearance: colorless, cle..."
10881,./test_hashed/form/9807fe93-30d5-422b-8844-fd1...,pull mailfile request to: bagby rjr-10301 july...
10882,./test_hashed/handwritten/e7eac2d5-364d-4f82-b...,~tclecen 8/23/38 saj co z & rh cam bt meaured ...


In [100]:
from langdetect import detect
detect(df_train.iloc[3495]['text'])

'en'

In [101]:
df_train.iloc[3495]['text']

'nav 03 03: [iam rhilip mrris [ra hk 85228263807 p.213 61 - ture conn 199541.237-219 tobacco control n inteanattonal u a na l editorials the ncws on tobacco control: time to bring the background into the foreground ainst baceroundnfdecllobur iobicco ue2ud rencr ecintionkfarerimple huranuentk incon atered 047 jb7 poncyc cbini cdt ect dilcait tisk [ac02. 1c? qoo aoektn 1e2*purtt clpicn tobacco bucs bom tucmadc muldzie gy l;cren 15999 _mdio teletbiod vuucb ljcte fan_or prevengnt heurubeaie acludddi ell9c erneninc ciele trc_tifoui rcuictcn [norouc inicvcnhon hdldas 1dccd hundrd lhousinda dotg_ suc lncmecantad gudfc rkf1ctoh onl\' nledotubsa livc fportars clly dlunmed commntd wu thc talucdoni di0 #cn ccjuo rcupi n-o cphemen: 40m: the mom5 polene 1n0 fundiod lagteaetlun ptani iepont err tocad eo10d2 lktort fobaeco edmd ohncome te mijor lul-gmunil mdohnk cl  portful dt4ac-gmic tcltuulen collmcntrty_ looz comun jimx meda 2g drolonzco cpjode preaddencbl can lidjc: bob amokne compicd dficai comn

In [121]:
sample = df_train.head()

In [122]:
sample['language'] = sample['text'].apply(lambda x: detect(x))

C:\Users\Wiktor\AppData\Local\Temp\ipykernel_15556\2992993266.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['language'] = sample['text'].apply(lambda x: detect(x))


In [114]:
spell1 = Speller(lang="pl", only_replacements=True)
spell2 = Speller(lang='en', only_replacements=True)
# sample[sample['language'] == 'pl']['text'] = sample[sample['language'] == 'pl']['text'].apply(lambda x: spell(x))

In [127]:
sample['text'] = np.where(sample['language'] == 'pl', sample['text'].apply(lambda x: spell1(x)), sample['text'].apply(lambda x: spell2(x)))

C:\Users\Wiktor\AppData\Local\Temp\ipykernel_15556\2576813700.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['text'] = np.where(sample['language'] == 'pl', sample['text'].apply(lambda x: spell1(x)), sample['text'].apply(lambda x: spell2(x)))


In [130]:
from autocorrect import Speller
def autocorrect_words(df, text_col: str) -> pd.DataFrame:
    languages = set(df['language'])
    spell_en = Speller(lang='en', only_replacements=True)
    spell_pl = Speller(lang='pl', only_replacements=True)
    df_pl = df[df['language'] == 'pl']
    df_en = df[df['language'] == 'en']
    df_pl['text'] = df_pl['text'].apply(lambda x: spell_pl(x))
    df_en['text'] = df_en['text'].apply(lambda x: spell_en(x))
    return pd.concat([df_pl, df_en])

In [ ]:
autocorrect_words()